In [2]:
# Descarga tu repositorio desde GitHub
!git clone https://github.com/Wmarbar/CProgram_incidencias.git

# Cambia de directorio para acceder a los archivos
%cd CProgram_incidencias

# Verifica que los archivos estén
!ls


Cloning into 'CProgram_incidencias'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12 (delta 1), reused 8 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 1.58 MiB | 5.72 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/CProgram_incidencias/CProgram_incidencias
 constructor4_proyecto_II.ipynb   README.md
 input				 'Trabajo-CalidadDeDatos Ver4-2.pdf'


# __PROGRAMACIÓN PARA ANALÍTICA DE DATOS__

<img src="https://github.com/WMARBAR/CProgram_incidencias/blob/main/logo.png?raw=1" alt="Logo Universidad Central" width="300" />

> ## __PROYECTO: Estimación Área bajo la curva__

> ## Profesor: Jorge Victorino [jvictorinog@ucentral.edu.co](jvictorinog@ucentral.edu.co)<br>
> ## Estudiante: Sergio Andrés Sánchez Cárdenas [correo@ucentral.edu.co](mailto:correo@ucentral.edu.co)<br>
> ## Estudiante: Wilson Felipe Mártinez Barrantes [wmartinezb1@ucentral.edu.co](jvictorinog@ucentral.edu.co)<br>
> ## Grupo: 5 <br>
> ### Facultad de Ingeniería y Ciencias Básicas <br>
> ### Universidad Central <br>
> <br>

---


In [3]:
import pandas as pd
import numpy as np

In [4]:
# Cargar archivos
df_incidentes = pd.read_excel("./input/incidentes1.xlsx")

In [5]:
nulos_totales_2 = df_incidentes.isnull().sum()

In [6]:
nulos_totales_2

,0
Incidente,0
Estructura,3453
Tipo incidente,33711
Estado,53
Prioridad,1655
Creacion,0
Inicio,0
Cierre,2279


In [7]:
df_incidentes.shape

(38251, 8)

In [8]:
df_incidentes.columns

Index(['Incidente', 'Estructura', 'Tipo incidente', 'Estado', 'Prioridad',
       'Creacion', 'Inicio', 'Cierre'],
      dtype='object')

In [9]:
df_incidentes.rename(columns={
    'Incidente': 'Codigo_Incidente',
    'Estructura': 'Codigo_Estructura',
    'Tipo incidente': 'Tipo_incidente',
    'Estado': 'Estado_Siglas',
    'Prioridad': 'Prioridad',
    'Creacion': 'Fecha_Creacion',
    'Inicio': 'Fecha_Inicio',
    'Cierre': 'Fecha_Cierre'
}, inplace=True)

In [10]:
df_incidentes

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre
0,1888349,19885059.0,NaN,INIC,Mes,2003-05-18,2003-05-18,2003-10-01
1,1888338,19797979.0,NaN,INIC,Semestre,2003-05-19,2003-05-19,2003-08-16
2,1888336,19798855.0,NaN,CCLI,Semana,2003-05-21,2003-05-21,2003-06-02
3,1888122,19892782.0,NaN,INIC,Semana,2003-05-28,2003-05-28,2003-06-02
4,1888094,19862532.0,NaN,PROY,Semestre,2003-05-29,2003-07-30,2005-08-06
...,...,...,...,...,...,...,...,...
38246,1012607,19878276.0,NaN,INIC ENPT,Año,2017-05-28,2018-05-03,NaT
38247,1012596,NaN,NaN,INIC ENPT,Año,2017-05-28,2018-09-19,2018-09-28
38248,1012558,19877965.0,NaN,INIC ENPT,Dos años,2017-05-28,2018-12-27,2018-12-27
38249,1012557,19877964.0,NaN,INIC ENPT,Dos años,2017-05-28,2018-06-23,2018-07-07


In [11]:
import unicodedata

# Función para quitar tildes y convertir a mayúsculas
def normalizar_texto(texto):
    if pd.isna(texto):
        return texto
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return texto.upper().strip()

In [12]:
df_incidentes['Tipo_incidente'] = df_incidentes['Tipo_incidente'].apply(normalizar_texto)

In [13]:
df_incidentes.isnull().any()


,0
Codigo_Incidente,False
Codigo_Estructura,True
Tipo_incidente,True
Estado_Siglas,True
Prioridad,True
Fecha_Creacion,False
Fecha_Inicio,False
Fecha_Cierre,True


In [14]:
df_incidentes.columns

Index(['Codigo_Incidente', 'Codigo_Estructura', 'Tipo_incidente',
       'Estado_Siglas', 'Prioridad', 'Fecha_Creacion', 'Fecha_Inicio',
       'Fecha_Cierre'],
      dtype='object')

In [15]:
df_incidentes[df_incidentes['Estado_Siglas'].isna()== True]

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre
187,1874539,19878539.0,NaN,NaN,Semestre,2003-11-06,2003-11-06,2010-04-11
1317,1813405,NaN,NaN,NaN,Año,2005-11-11,2005-11-11,2008-09-26
2369,1769654,19798765.0,NaN,NaN,Mes,2007-02-18,2007-02-18,2009-11-07
2371,1769651,19798129.0,NaN,NaN,Mes,2007-02-18,2007-02-18,2008-12-04
2372,1769650,19798128.0,NaN,NaN,Mes,2007-02-18,2007-02-18,2008-12-04
2373,1769646,19798127.0,NaN,NaN,Mes,2007-02-18,2007-02-18,2008-12-04
2465,1767763,NaN,NaN,NaN,Semestre,2006-12-29,2007-03-25,2008-12-04
2684,1762606,NaN,NaN,NaN,Tres años,2007-05-29,2007-05-30,2010-10-20
2688,1762574,NaN,NaN,NaN,Tres años,2007-05-29,2007-05-30,2010-10-20
2768,1759504,19884883.0,NaN,NaN,Año,2007-05-26,2007-06-26,2008-11-09


In [16]:
# Eliminar filas donde 'Estado_Siglas' sea NaN
df_incidentes.dropna(subset=['Estado_Siglas'], inplace=True)

In [17]:
df_incidentes

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre
0,1888349,19885059.0,NaN,INIC,Mes,2003-05-18,2003-05-18,2003-10-01
1,1888338,19797979.0,NaN,INIC,Semestre,2003-05-19,2003-05-19,2003-08-16
2,1888336,19798855.0,NaN,CCLI,Semana,2003-05-21,2003-05-21,2003-06-02
3,1888122,19892782.0,NaN,INIC,Semana,2003-05-28,2003-05-28,2003-06-02
4,1888094,19862532.0,NaN,PROY,Semestre,2003-05-29,2003-07-30,2005-08-06
...,...,...,...,...,...,...,...,...
38246,1012607,19878276.0,NaN,INIC ENPT,Año,2017-05-28,2018-05-03,NaT
38247,1012596,NaN,NaN,INIC ENPT,Año,2017-05-28,2018-09-19,2018-09-28
38248,1012558,19877965.0,NaN,INIC ENPT,Dos años,2017-05-28,2018-12-27,2018-12-27
38249,1012557,19877964.0,NaN,INIC ENPT,Dos años,2017-05-28,2018-06-23,2018-07-07


In [18]:
# Lista de siglas que te interesan
ESTADOS_INTERES = ['GPRE', 'PRER', 'TAMB', 'GEAM', 'AMPO', 'GERE']

# Función que detecta si hay coincidencia con ESTADOS_INTERES
def contiene_estado_interes(estado_str):
    palabras = estado_str.split()  # divide el string en palabras por espacios
    for palabra in palabras:
        if palabra in ESTADOS_INTERES:
            return True
    return False

# Aplicar la función a la columna del DataFrame
df_incidentes['Estado_de_interes'] = df_incidentes['Estado_Siglas'].apply(contiene_estado_interes)

# Filtrar solo los que contienen coincidencias
df_filtrado = df_incidentes[df_incidentes['Estado_de_interes'] == True]


In [19]:
df_filtrado

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes
19,1642746,19892454.0,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-02-24,2011-08-12,True
34,1642746,19892454.0,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True
35,2403089,19892490.0,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True
39,2403089,19892490.0,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True
49,2403088,19892490.0,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True
...,...,...,...,...,...,...,...,...,...
20071,2480332,19894578.0,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True
20072,2480332,19894578.0,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2016-07-30,2017-12-13,True
20073,2480332,19894578.0,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True
20074,2577091,19895073.0,INVASION/EXPLANACION,CTER COND GEAM CO19 GERE,Seis años,2014-07-18,2017-10-20,2018-12-27,True


In [20]:
df_filtrado.reset_index(drop=True, inplace=True)

In [21]:
df_filtrado

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes
0,1642746,19892454.0,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-02-24,2011-08-12,True
1,1642746,19892454.0,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True
2,2403089,19892490.0,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True
3,2403089,19892490.0,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True
4,2403088,19892490.0,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True
...,...,...,...,...,...,...,...,...,...
4535,2480332,19894578.0,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True
4536,2480332,19894578.0,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2016-07-30,2017-12-13,True
4537,2480332,19894578.0,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True
4538,2577091,19895073.0,INVASION/EXPLANACION,CTER COND GEAM CO19 GERE,Seis años,2014-07-18,2017-10-20,2018-12-27,True


In [22]:
df_filtrado.dtypes

,0
Codigo_Incidente,int64
Codigo_Estructura,float64
Tipo_incidente,object
Estado_Siglas,object
Prioridad,object
Fecha_Creacion,datetime64[ns]
Fecha_Inicio,datetime64[ns]
Fecha_Cierre,datetime64[ns]
Estado_de_interes,bool


In [23]:
df_filtrado['Codigo_Estructura'] = df_filtrado['Codigo_Estructura'].astype('Int64').astype(str)


<ipython-input-23-64c319af0bcb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['Codigo_Estructura'] = df_filtrado['Codigo_Estructura'].astype('Int64').astype(str)


In [24]:
df_filtrado

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes
0,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-02-24,2011-08-12,True
1,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True
2,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True
3,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True
4,2403088,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True
...,...,...,...,...,...,...,...,...,...
4535,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True
4536,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2016-07-30,2017-12-13,True
4537,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True
4538,2577091,19895073,INVASION/EXPLANACION,CTER COND GEAM CO19 GERE,Seis años,2014-07-18,2017-10-20,2018-12-27,True


In [25]:
#df_filtrado.to_excel('df_filtrado.xlsx', index=False)

In [26]:
df_filtrado

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes
0,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-02-24,2011-08-12,True
1,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True
2,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True
3,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True
4,2403088,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True
...,...,...,...,...,...,...,...,...,...
4535,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True
4536,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2016-07-30,2017-12-13,True
4537,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True
4538,2577091,19895073,INVASION/EXPLANACION,CTER COND GEAM CO19 GERE,Seis años,2014-07-18,2017-10-20,2018-12-27,True


In [27]:
df_filtrado.columns

Index(['Codigo_Incidente', 'Codigo_Estructura', 'Tipo_incidente',
       'Estado_Siglas', 'Prioridad', 'Fecha_Creacion', 'Fecha_Inicio',
       'Fecha_Cierre', 'Estado_de_interes'],
      dtype='object')

In [28]:
import pandas as pd

# Asegúrate de que las columnas están en formato datetime
df_filtrado['Fecha_Creacion'] = pd.to_datetime(df_filtrado['Fecha_Creacion'])
df_filtrado['Fecha_Inicio'] = pd.to_datetime(df_filtrado['Fecha_Inicio'])
df_filtrado['Fecha_Cierre'] = pd.to_datetime(df_filtrado['Fecha_Cierre'])

# Crear la columna 'dias_creacion_inicio'
df_filtrado['dias_creacion_inicio'] = (df_filtrado['Fecha_Inicio'] - df_filtrado['Fecha_Creacion']).dt.days

# También puedes añadir otras dos columnas si lo necesitas:
df_filtrado['dias_inicio_cierre'] = (df_filtrado['Fecha_Cierre'] - df_filtrado['Fecha_Inicio']).dt.days
df_filtrado['dias_creacion_cierre'] = (df_filtrado['Fecha_Cierre'] - df_filtrado['Fecha_Creacion']).dt.days


<ipython-input-28-699f261f47c8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['Fecha_Creacion'] = pd.to_datetime(df_filtrado['Fecha_Creacion'])
<ipython-input-28-699f261f47c8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['Fecha_Inicio'] = pd.to_datetime(df_filtrado['Fecha_Inicio'])
<ipython-input-28-699f261f47c8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [29]:
df_filtrado

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes,dias_creacion_inicio,dias_inicio_cierre,dias_creacion_cierre
0,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-02-24,2011-08-12,True,0,534.0,534.0
1,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True,30,504.0,534.0
2,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True,43,NaN,NaN
3,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True,0,NaN,NaN
4,2403088,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4535,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0
4536,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2016-07-30,2017-12-13,True,0,501.0,501.0
4537,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0
4538,2577091,19895073,INVASION/EXPLANACION,CTER COND GEAM CO19 GERE,Seis años,2014-07-18,2017-10-20,2018-12-27,True,1190,433.0,1623.0


In [30]:
df_filtrado['Codigo_Estructura'] = df_filtrado['Codigo_Estructura'].replace('<NA>', np.nan)

<ipython-input-30-4b12c212f182>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['Codigo_Estructura'] = df_filtrado['Codigo_Estructura'].replace('<NA>', np.nan)


In [31]:
df_filtrado

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes,dias_creacion_inicio,dias_inicio_cierre,dias_creacion_cierre
0,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-02-24,2011-08-12,True,0,534.0,534.0
1,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True,30,504.0,534.0
2,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True,43,NaN,NaN
3,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True,0,NaN,NaN
4,2403088,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4535,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0
4536,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2016-07-30,2017-12-13,True,0,501.0,501.0
4537,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0
4538,2577091,19895073,INVASION/EXPLANACION,CTER COND GEAM CO19 GERE,Seis años,2014-07-18,2017-10-20,2018-12-27,True,1190,433.0,1623.0


In [32]:
df_limpio = df_filtrado.dropna(subset=['Codigo_Estructura'])

In [33]:
df_limpio

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes,dias_creacion_inicio,dias_inicio_cierre,dias_creacion_cierre
0,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-02-24,2011-08-12,True,0,534.0,534.0
1,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True,30,504.0,534.0
2,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True,43,NaN,NaN
3,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True,0,NaN,NaN
4,2403088,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4535,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0
4536,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2016-07-30,2017-12-13,True,0,501.0,501.0
4537,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0
4538,2577091,19895073,INVASION/EXPLANACION,CTER COND GEAM CO19 GERE,Seis años,2014-07-18,2017-10-20,2018-12-27,True,1190,433.0,1623.0


In [34]:
df_limpio['llave_incidente_estructura'] = df_limpio['Codigo_Incidente'].astype(str) + "_" + df_limpio['Codigo_Estructura'].astype(str)


<ipython-input-34-b0d469bce7c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['llave_incidente_estructura'] = df_limpio['Codigo_Incidente'].astype(str) + "_" + df_limpio['Codigo_Estructura'].astype(str)


In [35]:
df_limpio

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes,dias_creacion_inicio,dias_inicio_cierre,dias_creacion_cierre,llave_incidente_estructura
0,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-02-24,2011-08-12,True,0,534.0,534.0,1642746_19892454
1,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True,30,504.0,534.0,1642746_19892454
2,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True,43,NaN,NaN,2403089_19892490
3,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True,0,NaN,NaN,2403089_19892490
4,2403088,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-06-27,NaT,True,0,NaN,NaN,2403088_19892490
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4535,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0,2480332_19894578
4536,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2016-07-30,2017-12-13,True,0,501.0,501.0,2480332_19894578
4537,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0,2480332_19894578
4538,2577091,19895073,INVASION/EXPLANACION,CTER COND GEAM CO19 GERE,Seis años,2014-07-18,2017-10-20,2018-12-27,True,1190,433.0,1623.0,2577091_19895073


In [36]:
df_filtrado = df_limpio[df_limpio['dias_creacion_inicio'] != 0]

In [37]:
df_filtrado

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes,dias_creacion_inicio,dias_inicio_cierre,dias_creacion_cierre,llave_incidente_estructura
1,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True,30,504.0,534.0,1642746_19892454
2,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True,43,NaN,NaN,2403089_19892490
5,2403088,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2019-02-27,NaT,True,245,NaN,NaN,2403088_19892490
7,2403087,19892489,VEGETACION,INIC GEAM GPRE,Dos años,2018-06-27,2018-07-19,NaT,True,22,NaN,NaN,2403087_19892489
9,2403092,19891964,VEGETACION,INIC ENPT GEAM PRER,Dos años,2018-06-27,2018-07-07,NaT,True,10,NaN,NaN,2403092_19891964
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4529,2368226,19894582,CONSTRUCCIONES,INIC GEAM GERE,Dos años,2019-04-14,2019-04-17,NaT,True,3,NaN,NaN,2368226_19894582
4534,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0,2480332_19894578
4535,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0,2480332_19894578
4537,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0,2480332_19894578


In [38]:
df_final = df_filtrado.drop_duplicates(subset='llave_incidente_estructura', keep='first')

In [39]:
df_final

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes,dias_creacion_inicio,dias_inicio_cierre,dias_creacion_cierre,llave_incidente_estructura
1,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True,30,504.0,534.0,1642746_19892454
2,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True,43,NaN,NaN,2403089_19892490
5,2403088,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2019-02-27,NaT,True,245,NaN,NaN,2403088_19892490
7,2403087,19892489,VEGETACION,INIC GEAM GPRE,Dos años,2018-06-27,2018-07-19,NaT,True,22,NaN,NaN,2403087_19892489
9,2403092,19891964,VEGETACION,INIC ENPT GEAM PRER,Dos años,2018-06-27,2018-07-07,NaT,True,10,NaN,NaN,2403092_19891964
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4521,2525455,19894582,INVASION/EXPLANACION,INIC REAN GERE,Dos años,2015-07-28,2017-07-28,2017-07-28,True,731,0.0,731.0,2525455_19894582
4522,2480051,19894582,CONSTRUCCIONES,INIC GERE TERC,Tres años,2016-08-03,2017-12-02,2017-12-09,True,486,7.0,493.0,2480051_19894582
4527,2368226,19894582,CONSTRUCCIONES,INIC GEAM GERE,Dos años,2019-04-14,2019-04-17,NaT,True,3,NaN,NaN,2368226_19894582
4534,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0,2480332_19894578


In [40]:
df_final.columns

Index(['Codigo_Incidente', 'Codigo_Estructura', 'Tipo_incidente',
       'Estado_Siglas', 'Prioridad', 'Fecha_Creacion', 'Fecha_Inicio',
       'Fecha_Cierre', 'Estado_de_interes', 'dias_creacion_inicio',
       'dias_inicio_cierre', 'dias_creacion_cierre',
       'llave_incidente_estructura'],
      dtype='object')

In [41]:
nulos_totales = df_final.isnull().sum()

In [42]:
nulos_totales

,0
Codigo_Incidente,0
Codigo_Estructura,0
Tipo_incidente,0
Estado_Siglas,0
Prioridad,2
Fecha_Creacion,0
Fecha_Inicio,0
Fecha_Cierre,330
Estado_de_interes,0
dias_creacion_inicio,0


In [43]:
df_final['Tipo_incidente'] = df_final['Tipo_incidente'].fillna('SIN_DATO')

<ipython-input-43-cc626e5a1b90>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Tipo_incidente'] = df_final['Tipo_incidente'].fillna('SIN_DATO')


In [44]:
df_final

,Codigo_Incidente,Codigo_Estructura,Tipo_incidente,Estado_Siglas,Prioridad,Fecha_Creacion,Fecha_Inicio,Fecha_Cierre,Estado_de_interes,dias_creacion_inicio,dias_inicio_cierre,dias_creacion_cierre,llave_incidente_estructura
1,1642746,19892454,OBRAS,INIC ENPT GERE,Año,2010-02-24,2010-03-26,2011-08-12,True,30,504.0,534.0,1642746_19892454
2,2403089,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2018-08-09,NaT,True,43,NaN,NaN,2403089_19892490
5,2403088,19892490,VEGETACION,INIC TAMB GERE,Dos años,2018-06-27,2019-02-27,NaT,True,245,NaN,NaN,2403088_19892490
7,2403087,19892489,VEGETACION,INIC GEAM GPRE,Dos años,2018-06-27,2018-07-19,NaT,True,22,NaN,NaN,2403087_19892489
9,2403092,19891964,VEGETACION,INIC ENPT GEAM PRER,Dos años,2018-06-27,2018-07-07,NaT,True,10,NaN,NaN,2403092_19891964
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4521,2525455,19894582,INVASION/EXPLANACION,INIC REAN GERE,Dos años,2015-07-28,2017-07-28,2017-07-28,True,731,0.0,731.0,2525455_19894582
4522,2480051,19894582,CONSTRUCCIONES,INIC GERE TERC,Tres años,2016-08-03,2017-12-02,2017-12-09,True,486,7.0,493.0,2480051_19894582
4527,2368226,19894582,CONSTRUCCIONES,INIC GEAM GERE,Dos años,2019-04-14,2019-04-17,NaT,True,3,NaN,NaN,2368226_19894582
4534,2480332,19894578,CONSTRUCCIONES,INIC GERE,Dos años,2016-07-30,2017-12-03,2017-12-13,True,491,10.0,501.0,2480332_19894578


In [45]:
df_filtrado.dtypes

,0
Codigo_Incidente,int64
Codigo_Estructura,object
Tipo_incidente,object
Estado_Siglas,object
Prioridad,object
Fecha_Creacion,datetime64[ns]
Fecha_Inicio,datetime64[ns]
Fecha_Cierre,datetime64[ns]
Estado_de_interes,bool
dias_creacion_inicio,int64


In [46]:
#df_final.to_excel('df_limpio_2.xlsx', index=False)

In [47]:
# TESTEO DE GUARDADO REPOSITORIO